In [11]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10,6)

In [ ]:
import glob

# df = pd.read_csv('../data/raw/euro-2024-2025/B1.csv')
csv_2024 = glob.glob('../data/raw/euro-2024-2025/*.csv')
csv_2023 = glob.glob('../data/raw/euro-2023-2024/*.csv')
csv_2022 = glob.glob('../data/raw/euro-2022-2023/*.csv')
csv_2021 = glob.glob('../data/raw/euro-2021-2022/*.csv')
csv_2020 = glob.glob('../data/raw/euro-2020-2021/*.csv')


# Div = League Division
# Date = Match Date (dd/mm/yy)
# Time = Time of match kick off
# HomeTeam = Home Team
# AwayTeam = Away Team
# FTHG and HG = Full Time Home Team Goals
# FTAG and AG = Full Time Away Team Goals
# FTR and Res = Full Time Result (H=Home Win, D=Draw, A=Away Win)
# HTHG = Half Time Home Team Goals
# HTAG = Half Time Away Team Goals
# HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)

# Match Statistics (where available)
# Attendance = Crowd Attendance
# Referee = Match Referee
# HS = Home Team Shots
# AS = Away Team Shots
# HST = Home Team Shots on Target
# AST = Away Team Shots on Target
# HHW = Home Team Hit Woodwork
# AHW = Away Team Hit Woodwork
# HC = Home Team Corners
# AC = Away Team Corners
# HF = Home Team Fouls Committed
# AF = Away Team Fouls Committed
# HFKC = Home Team Free Kicks Conceded
# AFKC = Away Team Free Kicks Conceded
# HO = Home Team Offsides
# AO = Away Team Offsides
# HY = Home Team Yellow Cards
# AY = Away Team Yellow Cards
# HR = Home Team Red Cards
# AR = Away Team Red Cards
# HBP = Home Team Bookings Points (10 = yellow, 25 = red)
# ABP = Away Team Bookings Points (10 = yellow, 25 = red)


In [73]:

def load_csv_files(file_list, index):
    dataframes = pd.DataFrame()
    for file in file_list:
        try:
            non_ascii_pattern = r'[^\x00-\x7F]+'
            df_temp = pd.read_csv(file, encoding='latin1')
            for col in df_temp.select_dtypes(include='object').columns:
                df_temp[col] = df_temp[col].str.replace(non_ascii_pattern, '', regex=True)
            dataframes = pd.concat([dataframes, df_temp], ignore_index=True)
        except Exception as e:
            print(f"Error reading at index {index} for file {file}: {e}")
    return dataframes


In [136]:
df1=df2=df3=df4=df5=pd.DataFrame()

for i, csv in enumerate([csv_2024, csv_2023, csv_2022, csv_2021, csv_2020], start=1):
    vars()[f'df{i}'] = load_csv_files(csv, i)

In [137]:
for i in range(1, 6):
    print(f"DataFrame df{i} shape: {vars()[f'df{i}'].shape}")  # Print the shape of each DataFrame

df1['Div'] = df1['ï»¿Div']
df1.drop('ï»¿Div', axis=1, inplace=True)

DataFrame df1 shape: (7681, 123)
DataFrame df2 shape: (7800, 106)
DataFrame df3 shape: (7829, 106)
DataFrame df4 shape: (7822, 108)
DataFrame df5 shape: (7648, 106)


In [138]:
df1.columns.to_list()  

['Date',
 'Time',
 'HomeTeam',
 'AwayTeam',
 'FTHG',
 'FTAG',
 'FTR',
 'HTHG',
 'HTAG',
 'HTR',
 'HS',
 'AS',
 'HST',
 'AST',
 'HF',
 'AF',
 'HC',
 'AC',
 'HY',
 'AY',
 'HR',
 'AR',
 'B365H',
 'B365D',
 'B365A',
 'BWH',
 'BWD',
 'BWA',
 'BFH',
 'BFD',
 'BFA',
 'PSH',
 'PSD',
 'PSA',
 'WHH',
 'WHD',
 'WHA',
 '1XBH',
 '1XBD',
 '1XBA',
 'MaxH',
 'MaxD',
 'MaxA',
 'AvgH',
 'AvgD',
 'AvgA',
 'BFEH',
 'BFED',
 'BFEA',
 'B365>2.5',
 'B365<2.5',
 'P>2.5',
 'P<2.5',
 'Max>2.5',
 'Max<2.5',
 'Avg>2.5',
 'Avg<2.5',
 'BFE>2.5',
 'BFE<2.5',
 'AHh',
 'B365AHH',
 'B365AHA',
 'PAHH',
 'PAHA',
 'MaxAHH',
 'MaxAHA',
 'AvgAHH',
 'AvgAHA',
 'BFEAHH',
 'BFEAHA',
 'B365CH',
 'B365CD',
 'B365CA',
 'BWCH',
 'BWCD',
 'BWCA',
 'BFCH',
 'BFCD',
 'BFCA',
 'PSCH',
 'PSCD',
 'PSCA',
 'WHCH',
 'WHCD',
 'WHCA',
 '1XBCH',
 '1XBCD',
 '1XBCA',
 'MaxCH',
 'MaxCD',
 'MaxCA',
 'AvgCH',
 'AvgCD',
 'AvgCA',
 'BFECH',
 'BFECD',
 'BFECA',
 'B365C>2.5',
 'B365C<2.5',
 'PC>2.5',
 'PC<2.5',
 'MaxC>2.5',
 'MaxC<2.5',
 'AvgC>2.5',


In [149]:
# Collecting all stats that's relevant to the result

# Div = League Division
# Date = Match Date (dd/mm/yy)
# Time = Time of match kick off
# HomeTeam = Home Team
# AwayTeam = Away Team
# FTHG and HG = Full Time Home Team Goals
# FTAG and AG = Full Time Away Team Goals
# FTR and Res = Full Time Result (H=Home Win, D=Draw, A=Away Win)
# HTHG = Half Time Home Team Goals
# HTAG = Half Time Away Team Goals
# HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)
# Referee = Match Referee
# HST = Home Team Shots on Target
# AST = Away Team Shots on Target
# HC = Home Team Corners
# AC = Away Team Corners
# HR = Home Team Red Cards
# AR = Away Team Red Cards

relevant_columns = ['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HST', 'AST', 'HC', 'AC', 'HR', 'AR']

ft_stat_df1 = df1[relevant_columns]


In [155]:
# ft_stat_df1.shape
# ft_stat_df1.isna().sum()
ft_stat_df1[ft_stat_df1['HST'] == 0]

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HST,AST,HC,AC,HR,AR
1,B1,27/07/2024,17:15,Beerschot VA,Oud-Heverlee Leuven,0,0,D,0.0,0.0,D,0.0,2.0,2.0,6.0,0.0,0.0
4,B1,27/07/2024,15:00,Dender,St. Gilloise,0,0,D,0.0,0.0,D,0.0,3.0,4.0,6.0,0.0,0.0
35,B1,25/08/2024,15:00,St Truiden,St. Gilloise,0,0,D,0.0,0.0,D,0.0,2.0,3.0,8.0,1.0,0.0
56,B1,21/09/2024,15:00,Beerschot VA,St Truiden,0,3,A,0.0,1.0,A,0.0,11.0,4.0,7.0,0.0,0.0
94,B1,27/10/2024,17:30,Gent,Genk,0,2,A,0.0,0.0,D,0.0,3.0,1.0,6.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7230,F1,16/02/2025,19:45,Rennes,Lille,0,2,A,0.0,0.0,D,0.0,7.0,4.0,8.0,1.0,0.0
7263,F1,16/03/2025,16:15,Brest,Reims,0,0,D,0.0,0.0,D,0.0,2.0,4.0,4.0,1.0,0.0
7434,T1,03/11/2024,13:00,Bodrumspor,Alanyaspor,0,0,D,0.0,0.0,D,0.0,3.0,2.0,5.0,1.0,0.0
7542,T1,09/02/2025,16:00,Antalyaspor,Buyuksehyr,0,0,D,0.0,0.0,D,0.0,0.0,3.0,2.0,0.0,0.0
